In [41]:
#读取视频文件
def loadmkv(file,img_num=10):
    import cv2
    #标注处理
    labels_list = []
    i = 1
    with open(file + '_steering.csv', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            labels_list.append([float(row['wheel'])])
            if (i>=img_num):
                break
            i = i + 1
    #图像处理        
    cap = cv2.VideoCapture(file +'_front.mkv')
    ret = True
    frame_list = []
    i = 1 
    while(ret == True):
        ret, frame = cap.read()
        if (ret == True):
            
            frame_list.append(frame)
            del frame
            if (i>=img_num):
                break
            i = i + 1
            #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #cv2.imshow('frame',gray)
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    
    return (np.array(frame_list),np.array(labels_list))

In [42]:
import pickle
from os.path import isfile, isdir
import csv

epochs_path = '../epochs/'
outputf_path = '../output/'
for i in range(1,2):
    filename = epochs_path + 'epoch0' + str(i)
    outputfilename = outputf_path + 'epoch0' + str(i)
    if not isfile(outputfilename + '.p'):
        features,labels = loadmkv(filename,10)
        with open(outputfilename + '.p', 'wb') as outputfile:
            pickle.dump((features,labels), outputfile)
        del features,labels
        
print('load file completion')

load file completion


In [1]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        x,y = pickle.load(fo, encoding='bytes')
    return x,y

In [48]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing import image
import numpy as np
from keras.applications import ResNet50,VGG16
from keras.datasets import cifar10
import keras

x,y = unpickle('../output/epoch01.p')
print('x shape:', x.shape)
print('y shape:', y.shape)
y = keras.utils.to_categorical(y, num_classes)
print(y[0])
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
num_classes =10
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])
model = ResNet50(include_top=False, weights=None,input_shape=(720,1280,3))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x, y,
          epochs=1,
          batch_size=1)
#score = model.evaluate(x_test, y_test, batch_size=10)
print('Score:', score)

x shape: (10, 720, 1280, 3)
y shape: (10, 1)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]


ValueError: Error when checking target: expected avg_pool to have 4 dimensions, but got array with shape (10, 10)